In [0]:
year = 2017

In [0]:
batch = 5000
sample_num = 1 # starts from 1
start_num = (sample_num - 1) * batch
#start_num = 1319 # IMDB blocked me here!
end_num = (sample_num) *batch

In [1]:
"""
This was a  module
"""
#=========================== Libraries
from requests import get 
from bs4 import BeautifulSoup
import os
import time
import pandas as pd
import numpy as np
import re

#========================== Vars
home_url = 'https://www.imdb.com'


#========================== Functions
def Page_movie_catalog(base_url):
    response = None
    try:
        response = get(base_url)
    except:
        print("Not loaded "+ base_url)
        time.sleep(np.random.random()+10)
        #--------------- get the response
        try:
            response = Page_movie_catalog(base_url)
        except:
            pass

    return response
    

def Page_title_returner(html_soup):
    try:
        return html_soup.find_all('div', class_="title_bar_wrapper")
    except:
        return np.NaN

    
def Page_title_year(title_bar):
    try:
        return title_bar[0].h1.text.replace('(','').replace(')','').split('\xa0')        
    except:
        return np.NaN
        
        
        
def Page_certificate_picker(title_bar):
    certificate_set = ('G','PG','PG-13','R','NC-17','Not Rated')
    gen_info = title_bar[0].find('div', class_="subtext")
    if gen_info != None :
        try:
            certificate = str(gen_info).split('|')[0]
            if (certificate.startswith('<div class="subtext">') and certificate.endswith('<span class="ghost">')):
                certificate = certificate.replace('<div class="subtext">','').replace('<span class="ghost">','').replace('\n','').strip()  
            if (certificate in certificate_set):
                return certificate
            else:
                return np.NaN
        except:
            return np.NaN
    if get_info == None:
        return np.NaN
        
            
        
        
def Page_movie_duration(title_bar):
    try:
        return title_bar[0].find('time').text.replace('\n','').strip()
    except:
        return np.NaN        
        
        
        
def Page_genre(title_bar):
    try:
        genre_soup = title_bar[0].find('div', class_="subtext").find_all('a')
    except:
        genre_soup = []
        
    if genre_soup != []:
        genre = []
        for i in genre_soup:
            if i['href'].startswith("/search/title?genres"):
                genre.append(i.text)
        return genre
    else:
        return np.NaN   
        
        
        
        
def Page_release_date(title_bar):
    try:
        release_date = title_bar[0].find('a', attrs = {'title':"See more release dates"}).text.replace('\n','').strip()
    except:
        release_date = []
        
    if release_date != []:
        return release_date
    else:
        return np.NaN
        
        
        
def Page_DWS(html_soup):
    DWS = []
    pro = html_soup.find_all('div', class_='credit_summary_item')
    if pro != []:
        for i in pro:
            DWS.append((i.h4.text, [ii.text for ii in i.find_all('a')]))
        return DWS
    else:
        return np.NaN
    
    
def Page_popularity(html_soup):
    rev_bar = html_soup.find('div', class_="titleReviewBar")
    if rev_bar != None:
        pop = rev_bar.find_all('div', class_="titleReviewBarSubItem")
    else:
        return np.NaN
    
    if pop != []:
        if len(pop) == 1 :
            r1 = pop[0].span.span['class'][1].replace('Image',' ').split()
            r2 = re.findall(r'\s*(\w+)\s*\((\s*\w+\s*)\)\s*',pop[0].span.text.replace('\n',''))
        if len(pop) == 3 :
            r1 = pop[2].span.span['class'][1].replace('Image',' ').split()
            r2 = re.findall(r'\s*(\w+)\s*\((\s*\w+\s*)\)\s*',pop[2].span.text.replace('\n',''))
        return {r1[0]:r2[0][0], 'updown':str(r1[1]) + str(r2[0][1])}
    else:
        return np.NaN
        
        
def Page_review_user_critic(html_soup):
    rev_bar = html_soup.find('div', class_="titleReviewBar")
    if rev_bar != None:
        try:
            reviews = rev_bar.find('div', class_="titleReviewBarItem titleReviewbarItemBorder").span
            return reviews.text.replace('\n','').replace(',','').split('|')
        except:
            return np.NaN
        
        
def Page_meta_score(html_soup):
    try:
        rev_bar = html_soup.find('div', class_="titleReviewBar")
        metascore = rev_bar.find('div', class_="titleReviewBarItem")
        score = metascore.find('a', attrs ={'href':"criticreviews"})
        return score.text.replace('\n','').strip(), score.div['class'][1].split('_')[1] 
    except:
        return np.NaN
        
    
def Page_related_movie(html_soup):
    related = html_soup.find('div', class_="rec_const_picker")

    
    related_movie_title = []
    related_movie_url = []
    try:
        related_movies = related.find_all('a')[:-2]
        for related_movie in related_movies:
            #url
            try:
                related_movie_url.append(home_url+related_movie['href'])
            except:
                related_movie_url.append(np.NaN)
                
            # title
            try:
                related_movie_title.append(related_movie.img['title'])
            except:
                related_movie_title.append(np.NaN)

        return list(zip(related_movie_title, related_movie_url))

    except:
        related_movie_url.append(np.NaN)
        related_movie_title.append(np.NaN)


            
    
    
    
def Page_abstract(html_soup):
    try:
        abstract = html_soup.find('div', {'class':'article', 'id':'titleStoryLine'})
        return abstract.find('div', class_ = 'inline canwrap').span.text.strip()
    except:
        return np.NaN 
        
        
            
def Page_most_related_keywords(html_soup):
    try:
        abstract = html_soup.find('div', {'class':'article', 'id':'titleStoryLine'})
        keywords = abstract.firelated_movie_scorend_all('a')

        most_related_keywords = []
        for keyword in keywords:
            if re.findall(r'(\/search\/keyword\?\w+)',keyword['href']) != []:
                most_related_keywords.append(keyword.text)
        return most_related_keywords
    except:
        return np.NaN
    
    
    
def Page_all_keywords(movie_url):
    try:
        all_keywords = movie_url+"/keywords"
        response = Page_movie_catalog(all_keywords)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        tag_table = html_soup.find('table', class_="dataTable evenWidthTable2Col").text.replace('\n',' ').split('Yes No')
        all_keywords = []
        relevancy = []
        for cell in tag_table:
            all_keywords.append(cell.strip().split('     ')[0].replace("\'","'"))
            if re.findall(r'(\d+ of \d+) found this relevant',cell.strip()) != []:
                relevancy.append(re.findall(r'(\d+ of \d+) found this relevant',cell.strip().replace("\'","'")))
            else:
                relevancy.append(np.NaN)

        return list(zip(all_keywords, relevancy))
    except:
        return np.NaN
    
    
    
    
    
    
def Page_PG(html_soup):
    pd_guide_dict1 = []
    pd_guide_dict2 = []
    try:
        pg_items = html_soup.find_all('li', class_="ipl-inline-list__item")
        pg_guide_item = []
        pg_guide_number = []
        pg_severity = []
        for pg_item in pg_items:
            if (pg_item.a['href'][:9] == '#advisory' and pg_item.a['href'] != '#advisory-spoilers') :
                #print(pg_item.text)
                pg_guide_number.append(int(re.findall(r'\w+ \((\d+)\)',pg_item.text)[0]))
                pg_guide_item.append(pg_item.text.split("(")[0].strip())
                severity_level = html_soup.find_all('section', attrs = {'id':pg_item.a['href'].split('#')[1]})[0]
                severity_level = severity_level.find_all('div', class_ = "advisory-severity-vote__container ipl-zebra-list__item")[0]
                pg_severity.append(severity_level.span.text)


        pd_guide_dict1 = dict(zip(pg_guide_item, zip(pg_guide_number,pg_severity) ))
    except:
        pd_guide_dict1 = np.NaN
        

            #============== SPOILER
    try:
        spoiler_items = html_soup.find('section', attrs = {'id':"advisory-spoilers"})
        if spoiler_items != None:
            if (spoiler_items != None):
                item_in_spoiler = []

                for item in spoiler_items.find_all('h4', class_="ipl-list-title"):
                    item_in_spoiler.append(item.text.strip())
            else:
                pass

            my_items = spoiler_items.find_all('ul', class_="ipl-zebra-list")
            severiy_code = []
            severiy_num = []
            severiy_stat = []
            for (my_item,item) in  zip(my_items,item_in_spoiler):
                severiy_code.append(item)
                severiy_num.append(my_item.text.count("Edit")) 
                severiy_stat.append(np.NaN)

            pd_guide_dict2 = dict(zip(severiy_code, zip(severiy_num,severiy_stat) ))
    except:
        pd_guide_dict2 = np.NaN
    
    

        
    #========== 
        
    return pd_guide_dict1, pd_guide_dict2
    



def Page_certificate_detail(html_soup):
    MPAA = html_soup.find('tr', class_='ipl-zebra-list__item')
    try:
        certificate_reason = MPAA.find_all('td')
        if len(certificate_reason) == 2:
            return certificate_reason[1].text
        else: 
            return np.NaN
    except:
        return np.NaN
        
        
        
        
def Page_certificate_country(html_soup):
    try:
        certificate_country = html_soup.find_all('li', class_='ipl-inline-list__item')
        certificate_dict = dict()
        for j in certificate_country:
            if (j.a['href'].split('=')[0] == '/search/title?certificates'):
                d_object = j.a.text.split(":")
                certificate_dict[d_object[0]] = d_object[1]


        return certificate_dict
    except:
        return np.NaN



def list_join(mylist):
    out_str = ''
    for i in mylist:
        out_str += ':'+str(i)
    return out_str[1:]
    
    
def Movie_detail(html_soup):
    try:
        details = html_soup.find('div', class_= "article", attrs ={'id':"titleDetails"})
        details = details.find_all('div', class_='txt-block')
        d1 = []
        d2 = []
        for detail in details:
            detail = detail.text.replace('See more\xa0»','').replace('\n','').strip().split(':')
            if len(detail)<= 1:
                pass
            elif len(detail) == 2:
                d1.append(detail[0])
                d2.append(detail[1])
            elif len(detail) > 2:
                d1.append(detail[0])
                d2.append(list_join(detail[1:]))

        return dict(zip(d1,d2))
    except:
        return np.NaN
    
    
    
def star_name(movie_url):
    try:
        stars = pd.read_html(movie_url)
        stars = stars[0].drop([0,2],axis=1)
        stars.drop(0,axis=0,inplace=True)
        return stars
    except:
        return np.NaN
                    
def Poster_link(html_soup):
    try:
        poster = html_soup.find_all('div', class_="poster")[0]
        poster = poster.find('img')
        return poster.attrs['src']
    except:
        return np.NaN

    
def Page_related_movie_score(related_movie):
    related_movie_score = []
    try:
        for r in related_movie:
            response = Page_movie_catalog(r[1]+'ratings')
            film_id = (r[1].split('/')[-2])
            html_soup = BeautifulSoup(response.text, 'html.parser') 
            num = html_soup.find_all('div', class_="allText")[0].text.replace('\n','').replace(',','')
            score  = re.findall(r'(\d+)IMDb users have given a weighted average vote of\s+(\d+.\d+)', num)
            related_movie_score.append([score,film_id])

        return related_movie_score
    except:
        related_movie_score.append(np.nan)

In [ ]:
imdb_movie = "tt3315342"
movie_url = 'https://www.imdb.com/title/'+imdb_movie
response = Page_movie_catalog(movie_url)
html_soup = BeautifulSoup(response.text, 'html.parser') 

In [0]:
def Page_data_picker(imdb_movie):

    movie_url = 'https://www.imdb.com/title/'+imdb_movie
    response = Page_movie_catalog(movie_url)
    html_soup = BeautifulSoup(response.text, 'html.parser')   
    #============     title bar
    title_bar = Page_title_returner(html_soup)
    #====            functions use title bar
    movie_name, movie_year = Page_title_year(title_bar)
    certificate = Page_certificate_picker(title_bar)
    duration = Page_movie_duration(title_bar)
    genre = Page_genre(title_bar)


    #==== movie detail
    details = Movie_detail(html_soup)
    DWS = Page_DWS(html_soup)
    stars = star_name(movie_url)

    try:
        pop = Page_popularity(html_soup)
        pop_score = pop['popularity']
        pop_change = pop['updown']
    except:
        pop_score = np.NaN
        pop_change = np.NaN

    try:
        meta = Page_meta_score(html_soup)
        meta_score = meta[0]
        meta_favor = meta[1]
    except:
        meta_score = np.NaN
        meta_favor = np.NaN
        

    try:
        UserCritic_num = Page_review_user_critic(html_soup)
        user = UserCritic_num[0]
        critic = UserCritic_num[1]
    except:
        user = np.NaN
        critic = np.NaN


    related_movie = Page_related_movie(html_soup)

    related_movie_score = Page_related_movie_score(related_movie)

    
    abstract = Page_abstract(html_soup)

    most_related = Page_most_related_keywords(html_soup)
    all_related = Page_all_keywords(movie_url)


    #============== PG url
    pg_url = movie_url+"/parentalguide"
    response = Page_movie_catalog(pg_url)
    html_soup_PG = BeautifulSoup(response.text, 'html.parser')

    PGs = Page_PG(html_soup_PG)
    PG_detail = Page_certificate_detail(html_soup_PG)

    PG_Country = Page_certificate_country(html_soup_PG)

    #============== rating page
    movie_rating = movie_url +'/ratings'
    try:
        rating_page_table = pd.read_html(movie_rating)
        if len(rating_page_table)>= 2:
            rating_distro_vote_num = rating_page_table[0]
            rating_distro_age_num = rating_page_table[1]
    except:
        rating_distro_vote_num = np.NaN
        rating_distro_age_num = np.NaN

    try:
        stars = stars.to_dict()
    except: 
        stars = np.NaN   
    
    try:
        rating_distro_vote_num = rating_distro_vote_num.to_dict()
    except:
        rating_distro_vote_num = np.NaN

    try:
        rating_distro_age_num = rating_distro_age_num.to_dict()
    except:
        rating_distro_age_num = np.NaN
        
    poster_link = Poster_link(html_soup)

    page_dataframe = pd.DataFrame({'Title':[movie_name],
                                   'Year':[movie_year],
                                   'Movie detail':[details],
                                   'Certificate':[certificate],
                                   'Duration':[duration],
                                   'Genre':[genre],
                                   'DWS':[DWS],
                                   'Stars':[stars],
                                   'Popularity':[pop_score],
                                   'Popularity up or down':[pop_change],
                                   'Matascore':[meta_score],
                                   'Metafavor':[meta_favor],
                                   'User review':[user],
                                   'Critic review':[critic],
                                   'Related movie':[related_movie],
                                   'abstract':[abstract],
                                   'keywords':[most_related],
                                   'all keywords':[all_related],
                                   'PG':[PGs],
                                   'PG detail':[PG_detail],
                                   'PG Country':[PG_Country],
                                   'vote_distro':[rating_distro_vote_num],
                                   'age_of_voter_distro':[rating_distro_age_num],
                                   'Poster link':[poster_link],
                                   'Related movie score':[related_movie_score]
                                  })


    return page_dataframe


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
os.chdir("./IMDB%d/"%(year))

In [0]:
os.mkdir("IMDB_"+str(sample_num))
os.chdir("IMDB_"+str(sample_num))

In [0]:
movie_catalog = pd.read_csv('../movie_list_%d.csv'%(year))

In [0]:
file_path = 'movie_page_data___sample_num_'+str(sample_num)+'.csv'


dataframe_header  = pd.DataFrame(columns =['Title','Year','Movie detail','Certificate','Duration','Genre','DWS',
                                        'Stars','Popularity','Popularity up or down','Matascore','Metafavor',
                                        'User review','Critic review','Related movie','abstract','keywords',
                                        'all keywords','PG','PG detail','PG Country','vote_distro',
                                        'age_of_voter_distro','Poster link','Related movie score'])

with open(file_path, 'w') as f:
    dataframe_header.to_csv(f)



In [0]:
urls = movie_catalog['movie_url'].values

In [0]:
if end_num <= len(urls):
    for i, url in enumerate(urls[start_num:end_num]):
        url = url.split('/')[2]
        print(i,'  crawling movie',str(url))
        dataframe_of_movies = Page_data_picker(url)
        with open(file_path, 'a') as f:
            dataframe_of_movies.to_csv(f, header=False)
elif end_num > len(urls):
    for i,url in enumerate(urls[start_num:]):
        url = url.split('/')[2]
        print(i,'  crawling movie',str(url))
        dataframe_of_movies = Page_data_picker(url)
        with open(file_path, 'a') as f:
            dataframe_of_movies.to_csv(f, header=False)

0   crawling movie tt3315342
1   crawling movie tt3501632
2   crawling movie tt3896198
3   crawling movie tt0451279
4   crawling movie tt5013056
5   crawling movie tt2527336
6   crawling movie tt2250912
7   crawling movie tt5052448
8   crawling movie tt1396484
9   crawling movie tt1856101
10   crawling movie tt3890160
11   crawling movie tt5027774
12   crawling movie tt0974015
13   crawling movie tt5580390
14   crawling movie tt4425200
15   crawling movie tt2380307
16   crawling movie tt2283362
17   crawling movie tt2771200
18   crawling movie tt3731562
19   crawling movie tt4649466
20   crawling movie tt1790809
21   crawling movie tt2316204
22   crawling movie tt3450958
23   crawling movie tt1485796
24   crawling movie tt4925292
25   crawling movie tt5442430
26   crawling movie tt4630562
27   crawling movie tt3402236
28   crawling movie tt1219827
29   crawling movie tt5362988
30   crawling movie tt1972591
31   crawling movie tt1959563
32   crawling movie tt5109784
33   crawling movie 

AttributeError: ignored